In [47]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

ts_window = [3,6,9,12,18,24,30,36,48,60]


In [5]:
factor_portfolio_df = pd.read_csv(r'D:\Users\OneDrive\桌面\factor_research_Ashare\Size_BM_research\Size独立\monthly_reb\流通市值\SMB_factor_portfolio_5x1.csv')
char_rank_portfolio_df = pd.read_csv(r'D:\Users\OneDrive\桌面\factor_research_Ashare\Size_BM_research\Size独立\monthly_reb\流通市值\size_5groups_portfolio_return.csv')
factor_portfolio_df

,year,month,return
0,2005,2,0.026905
1,2005,3,-0.056524
2,2005,4,-0.109729
3,2005,5,0.083045
4,2005,6,-0.004473
...,...,...,...
179,2020,1,0.024322
180,2020,2,0.006187
181,2020,3,0.052489
182,2020,4,-0.049108


In [6]:
char_rank_portfolio_df

,year,month,code,return
0,2005,2,1,0.085373
1,2005,2,2,0.106170
2,2005,2,3,0.111629
3,2005,2,4,0.111460
4,2005,2,5,0.112278
...,...,...,...,...
915,2020,5,1,-0.006913
916,2020,5,2,0.016585
917,2020,5,3,0.008740
918,2020,5,4,0.024848


In [7]:
char_rank_portfolio_df = pd.merge(char_rank_portfolio_df,factor_portfolio_df,'left',['year','month'])

char_rank_portfolio_df

,year,month,code,return_x,return_y
0,2005,2,1,0.085373,0.026905
1,2005,2,2,0.106170,0.026905
2,2005,2,3,0.111629,0.026905
3,2005,2,4,0.111460,0.026905
4,2005,2,5,0.112278,0.026905
...,...,...,...,...,...
915,2020,5,1,-0.006913,0.057587
916,2020,5,2,0.016585,0.057587
917,2020,5,3,0.008740,0.057587
918,2020,5,4,0.024848,0.057587


In [51]:
def fun(subdf):
    def RollOls_params(indices, ycol, xcols):
        roll_df=subdf.loc[indices]
        result = sm.OLS(roll_df[ycol], roll_df[xcols]).fit()
        params = result.params.iloc[0]
        T_value = result.tvalues.iloc[0]
        tmp_para_result_df.loc[indices[-1],xcols[0]] = params
        tmp_T_result_df.loc[indices[-1],xcols[0]] = T_value
        return 0

    summary_df = subdf.loc[:,['year','month','code']]
    tmp_para_result_df = subdf[['return_y','index']].copy()
    tmp_T_result_df = subdf[['return_y','index']].copy()
    for window in ts_window:
        subdf['index'].rolling(window).apply(lambda indices:RollOls_params(indices, ycol=ycol,xcols=xcols))
        summary_df[f'window_{window}_para'] = tmp_para_result_df[xcols[0]].iloc[window-1:]
        summary_df[f'window_{window}_T'] = tmp_T_result_df[xcols[0]].iloc[window-1:]
    return summary_df
ycol = 'return_x'
xcols = ['return_y','const']
char_rank_portfolio_df['const'] = 1
char_rank_portfolio_df['index'] = char_rank_portfolio_df.index
ts_reg_result_df = char_rank_portfolio_df.groupby('code').apply(fun)
ts_reg_result_df

return_y   -0.058512
const      -0.024193
dtype: float64 return_y   -0.085080
const      -0.465378
dtype: float64
return_y   -0.285503
const      -0.044997
dtype: float64 return_y   -0.613539
const      -1.299397
dtype: float64
return_y   -0.423115
const      -0.027099
dtype: float64 return_y   -1.619005
const      -1.181072
dtype: float64
return_y   -0.032646
const       0.001034
dtype: float64 return_y   -0.089825
const       0.026414
dtype: float64
return_y    0.094575
const       0.017034
dtype: float64 return_y    0.692978
const       1.228296
dtype: float64
return_y    0.166766
const      -0.001787
dtype: float64 return_y    0.761330
const      -0.079698
dtype: float64
return_y    0.496230
const      -0.033078
dtype: float64 return_y    22.666372
const      -16.791012
dtype: float64
return_y   -0.403668
const      -0.000326
dtype: float64 return_y   -0.751233
const      -0.012983
dtype: float64
return_y   -0.630047
const       0.012032
dtype: float64 return_y   -0.780757
const   

,year,month,code,window_4_para,window_4_T
0,2005,2,1,NaN,NaN
1,2005,2,2,NaN,NaN
2,2005,2,3,NaN,NaN
3,2005,2,4,NaN,NaN
4,2005,2,5,NaN,NaN
...,...,...,...,...,...
915,2020,5,1,-0.783828,-2.176004
916,2020,5,2,-0.558109,-1.112517
917,2020,5,3,-0.358318,-1.114112
918,2020,5,4,-0.036475,-0.129472


In [49]:
ts_reg_result_df.to_csv(r'D:\Users\OneDrive\桌面\factor_research_Ashare\Size_BM_research\Size独立\monthly_reb\流通市值\时间序列回归研究\size组合时间序列回归结果多维.csv')